In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
from scipy.sparse import csr_matrix, csc_matrix

In [ ]:
# Étape 1 : Création du dataframe
def load_csv_to_dataframe(file_path):
    return pd.read_csv(file_path)
df_avant_traitement = load_csv_to_dataframe('logs_corrompu_label.csv')

In [ ]:
# Étape 2 : Création du job de prétraitement
def preprocess_data(file_path, save_job=True, job_file='preprocessing_pipeline.joblib'):
    df = pd.read_csv(file_path)

    # a. Enlever toutes les lignes où la Date est vide
    df = df.dropna(subset=['Date'])

    # b. Affecter à tous les Process "kernel" la valeur 10 à l'IdProcess quand il est NaN
    # df.loc[(df['Process'] == 'kernel') & (df['IdProcess'].isna()), 'IdProcess'] = 10

    # c. Limiter les colonnes à encoder
    max_unique_values = 1000
    label_column = 'Label'
    columns_to_encode = [col for col in df.columns.difference([label_column]) 
                         if df[col].nunique() <= max_unique_values]

    # d. Appliquer un OneHotEncoding sparse
    encoder = OneHotEncoder(sparse=True, handle_unknown='ignore')
    encoded_features = encoder.fit_transform(df[columns_to_encode])
    df_encode = pd.DataFrame.sparse.from_spmatrix(encoded_features, 
                                                  columns=encoder.get_feature_names_out(columns_to_encode))

    # e. Ajouter la colonne Label
    if label_column in df.columns:
        df_encode[label_column] = df[label_column].values

    # f. Sauvegarder le job de prétraitement
    if save_job:
        joblib.dump(encoder, job_file)

    return df_encode

df_encode = preprocess_data('logs_corrompu_label.csv')


In [ ]:
# Étape 3 : Entraînement du modèle avec RandomForest
def train_model_without_smote(df_encoded, label_column='Label', save_model=True, model_file='random_forest_model.joblib'):
    # Vérifier les valeurs manquantes dans la colonne cible
    if df_encoded[label_column].isna().sum() > 0:
        print(f"Attention : {df_encoded[label_column].isna().sum()} valeurs manquantes dans '{label_column}'.")
        df_encoded = df_encoded.dropna(subset=[label_column])  # Supprimer les lignes avec Label manquant

    X = df_encoded.drop(columns=[label_column])
    y = df_encoded[label_column]

    # Vérifier les types des données (convertir au besoin)
    y = pd.to_numeric(y, errors='coerce')  # S'assurer que y est numérique
    if y.isna().sum() > 0:
        print(f"Attention : {y.isna().sum()} étiquettes invalides après conversion en numérique.")
        df_encoded = df_encoded[~y.isna()]
        X = df_encoded.drop(columns=[label_column])
        y = df_encoded[label_column]

    # Vérification et conversion de la matrice encodée en csr_matrix
    if isinstance(X, csc_matrix):
        print("Conversion de X en csr_matrix...")
        X = X.tocsr()  # Conversion de csc_matrix en csr_matrix
    elif not isinstance(X, csr_matrix):
        print("Conversion de X en csr_matrix...")
        X = csr_matrix(X)  # Conversion explicite si nécessaire
    
    # Vérifier si la conversion a réussi
    if isinstance(X, csr_matrix):
        print(f"X est maintenant une matrice csr_matrix de type {type(X)}")
    else:
        print(f"X n'est pas une csr_matrix. Conversion en dense...")
        X = np.array(X)  # Conversion en dense si nécessaire
    
    # Appliquer un modèle Random Forest
    model = RandomForestClassifier(random_state=42, n_estimators=100)  # Ajustez les paramètres comme nécessaire
    try:
        model.fit(X, y)
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle : {e}")
        raise e

    # Calculer l'accuracy sur l'ensemble d'entraînement
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    print(f"Accuracy du modèle sur l'ensemble d'entraînement : {accuracy:.4f}")
    
    # Sauvegarder le modèle
    if save_model:
        joblib.dump(model, model_file)

    return model, accuracy

# Appel de la fonction d'entraînement
try:
    model, accuracy = train_model_without_smote(df_encode)
except Exception as e:
    print(f"Erreur lors de l'entraînement du modèle : {e}")


In [41]:
# Étape 4 : Test du modèle
def test_model(test_file, preprocessing_job='preprocessing_pipeline.joblib', model_file='random_forest_model.joblib'):
    # a. Charger le fichier de test
    df_all = pd.read_csv(test_file)

    # Vérifier si la colonne 'Label' existe dans le fichier de test
    if 'Label' not in df_all.columns:
        print("Avertissement : La colonne 'Label' n'existe pas dans le fichier de test. Une colonne fictive 'Label' a été ajoutée.")
        df_all['Label'] = 0  # Ajouter une colonne fictive 'Label' avec une valeur par défaut (0) si elle n'existe pas
    else:
        print("La colonne 'Label' existe dans le fichier de test.")

    # b. Charger le prétraitement
    encoder = joblib.load(preprocessing_job)

    # Synchroniser les colonnes (s'assurer que df_all a les bonnes colonnes pour l'encodage)
    columns_to_remove = set(df_all.columns) - set(encoder.feature_names_in_)
    df_all = df_all.drop(columns=columns_to_remove, errors='ignore')
    
    # Ajouter les colonnes manquantes pour que df_all corresponde à encoder.feature_names_in_
    missing_columns = set(encoder.feature_names_in_) - set(df_all.columns)
    for col in missing_columns:
        df_all[col] = 0  # Ou une autre valeur par défaut

    # Appliquer le prétraitement
    encoded_features = encoder.transform(df_all)

    # Vérification de la forme des données encodées
    print(f"Forme des données encodées : {encoded_features.shape}")
    
    # Vérifier si le résultat est dense ou sparse
    if hasattr(encoded_features, 'toarray'):
        encoded_features = encoded_features.toarray()

    # Si la forme des données est correcte, on peut créer un DataFrame
    df_all_encode = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out())
    
    # Supprimer la colonne encodée `Label` avant la prédiction
    if 'Label' in df_all_encode.columns:
        df_all_encode = df_all_encode.drop(columns=['Label'])

    # c. Charger le modèle
    model = joblib.load(model_file)

    # d. Prédire les anomalies
    predictions = model.predict(df_all_encode)

    # Renvoyer les lignes correspondant aux anomalies
    anomalies = df_all[predictions == 1]  # RandomForestClassifier marque les anomalies avec 1
    
    return anomalies

# Tester le modèle sur un fichier de test
anomalies = test_model('all_logs.csv')
print("Lignes comportant des anomalies :")
print(anomalies)


c:\Users\colin\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Attention : 39646 valeurs manquantes dans 'Label'.
Conversion de X en csr_matrix...
X est maintenant une matrice csr_matrix de type <class 'scipy.sparse._csr.csr_matrix'>
Accuracy du modèle sur l'ensemble d'entraînement : 0.9995
Avertissement : La colonne 'Label' n'existe pas dans le fichier de test. Une colonne fictive 'Label' a été ajoutée.
Forme des données encodées : (216815, 383)


c:\Users\colin\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  


Lignes comportant des anomalies :
         Hostname Process
164375  hilbert19    sshd
164376  hilbert19    sshd
164461  hilbert19    sshd
164462  hilbert19    sshd
164515  hilbert19    sshd
...           ...     ...
165332  hilbert19    sshd
165333  hilbert19    sshd
165334  hilbert19    sshd
165335  hilbert19    sshd
165336  hilbert19    sshd

[696 rows x 2 columns]
